In [1]:
pip install opencv-python==3.4.18.65

  Using cached opencv_python-3.4.18.65-cp36-abi3-macosx_10_15_x86_64.whl.metadata (18 kB)
Using cached opencv_python-3.4.18.65-cp36-abi3-macosx_10_15_x86_64.whl (44.3 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.9.0
ERROR: Cannot uninstall opencv-python 4.9.0, RECORD file not found. Hint: The package was installed by conda.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import pyautogui
import matplotlib.pyplot as plt
import apriltag
import argparse
import pygame as pg, pygamebg, math
from screeninfo import get_monitors
import tkinter as tk
import sys
pyautogui.FAILSAFE = False

screen_dist = 15.24 #distance from user to screen in cm
pg.init()

def angle_offset(screen_center, circle_center, cursor, screen_dist):
    
    x_dist = np.sqrt((circle_center[0] - screen_center[0])**2 + (circle_center[1] - screen_center[1])**2)
    x_dist = x_dist * 0.0264583333
    
    y_dist = np.sqrt((cursor[0] - circle_center[0])**2 + (cursor[1] - circle_center[1])**2)
    y_dist = y_dist * 0.0264583333
    
    alpha = np.sqrt((cursor[0] - screen_center[0])**2 + (cursor[1] - screen_center[1])**2)
    alpha = alpha * 0.0264583333
    
    d = np.sqrt(x_dist**2 + screen_dist**2)
    r = np.sqrt(alpha**2 + screen_dist**2)

    theta = math.acos((d**2 + r**2 - y_dist**2)/(2*d*r))
    theta = math.degrees(theta)
    
    return theta

def scaling(screen_center, circle_center, cursor):
    goal_distance_x = circle_center[0] - screen_center[0]
    goal_distance_y = circle_center[1] - screen_center[1]
    movement_distance_x = cursor[0] - screen_center[0]
    movement_distance_y = cursor[1] - screen_center[1]
    
    scaling_factor = (goal_distance_x / movement_distance_x, goal_distance_y/movement_distance_y)
    
    return scaling_factor #x,y tuple
            


def calibrate(screen_dist, x0, y0, perm_offset, xcoord, ycoord):
    root = tk.Tk()
    width = root.winfo_screenwidth()
    height = root.winfo_screenheight() #getting screen size

    pg.init()
    window = pg.display.set_mode((width, height))
    pg.display.set_caption("Calibration Window")

    
    circle_center = (width-100, height-100)
    screen_center = (width/2,height/2) 
        
        
    text = font.render('Move your cursor to the circle and press the letter w when done', True, pg.Color("black"))
    textRect = text.get_rect()
    # set the center of the rectangular object.
    textRect.center = (width/2, height/2+200)
    white = (255, 255, 255)
    
    found = 1
    eye_at_corner_x = 0
    eye_at_corner_y = 0
    while True:
        window.fill(white)
        pg.draw.circle(window, pg.Color("red"), (width-100,height-100), 10)
        window.blit(text, textRect)

        pyautogui.moveTo(xcoord - perm_offset[0], ycoord - perm_offset[1])

        for event in pg.event.get():
            if pg.key.get_pressed()[pg.K_w]:
                cursor_x, cursor_y = pg.mouse.get_pos()
                found = 0   
                break
            pg.display.update()
        if found == 0:
                break
                
    print(screen_center, circle_center, cursor_x, cursor_y)
    
    theta = angle_offset(screen_center, circle_center, (x0, y0), screen_dist)
    scale_factor = scaling(screen_center, circle_center, (cursor_x, cursor_y))
    
    return theta, scale_factor

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# init part
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

#***DEFAULT PARAMETERS***#

# Change thresholds (0 to 255) 
params.minThreshold = 0
params.maxThreshold = 255

#Area - area in terms of pixels
params.filterByArea = True
params.minArea = 300
params.maxArea = 10000
      
#Circularity - measures how close the blob to a circle
params.filterByCircularity = True
params.minCircularity = 0.5 #"cicularity" of a square = 0.785 , circle = 1.000
params.maxCircularity = 1.00 

#Color - 0 (select darker) to  255 (select lighter)
params.filterByColor = False
#params.minColor = 0 
#params.maxColor = 255 

#Convexity - (Area of the Blob / Area of it’s convex hull)
params.filterByConvexity = False
params.minConvexity = 0.87
params.maxConvexity = 1.00

#Inertia - how elongated a shape is, line = 0.000, circle = 1.000,
params.filterByInertia = False
params.minInertiaRatio = 0.95
params.maxInertiaRatio = 1.00

#Repeatability
params.minRepeatability=1

font2 = cv2.FONT_HERSHEY_SIMPLEX
font = pg.font.Font('freesansbold.ttf', 18)

# def blob_process(img, threshold, detector):
#     gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, img = cv2.threshold(gray_frame, threshold, 255, cv2.THRESH_BINARY)
#     img = cv2.erode(img, None, iterations=2)
#     img = cv2.dilate(img, None, iterations=4)
#     img = cv2.medianBlur(img, 5)
#     keypoints = detector.detect(img)
#     print(keypoints)
#     return keypoints


def nothing(x):
    pass

def main():
    screen_w, screen_h = pyautogui.size()
    eyeCap = cv2.VideoCapture(0)
    #screenCap = cv2.VideoCapture(1)
    
    #pcw = 'Parameter Control Window'
    #cv2.namedWindow(pcw)
    #slf = 'Screen Live Feed'
    #cv2.namedWindow(slf)
    image = 'Pupil Tracking Live Feed'
    cv2.namedWindow(image)
    
    threshold = 20
    areaParam = 1000
    circularityParam = 100
    colorParam = 20
    convexityParam = 100
    inertiaParam = 100
    
    
    cursorSwitch = 'Cursor OFF/ON'
    cv2.createTrackbar(cursorSwitch, image, 0, 1, nothing)
    
    iter = 1
    
    while True:
        #frame for eye
        _, eyeFrame = eyeCap.read()
        #frame for screen
       # _, screenFrame = screenCap.read()

        #convert to grayscale
        #gray_frame = cv2.cvtColor(screenFrame, cv2.COLOR_BGR2GRAY)

#         print("[INFO] detecting AprilTags...")
#         options = apriltag.DetectorOptions(families="tag36h11")
#         detector = apriltag.Detector(options)
#         results = detector.detect(gray_frame)
#         print("[INFO] {} total AprilTags detected".format(len(results)))

#         # loop over the AprilTag detection results
#         for r in results:
#             # extract the bounding box (x, y)-coordinates for the AprilTag
#             # and convert each of the (x, y)-coordinate pairs to integers
#             (ptA, ptB, ptC, ptD) = r.corners
#             ptB = (int(ptB[0]), int(ptB[1]))
#             ptC = (int(ptC[0]), int(ptC[1]))
#             ptD = (int(ptD[0]), int(ptD[1]))
#             ptA = (int(ptA[0]), int(ptA[1]))

#             # draw the bounding box of the AprilTag detection
#             cv2.line(screenFrame, ptA, ptB, (0, 255, 0), 2)
#             cv2.line(screenFrame, ptB, ptC, (0, 255, 0), 2)
#             cv2.line(screenFrame, ptC, ptD, (0, 255, 0), 2)
#             cv2.line(screenFrame, ptD, ptA, (0, 255, 0), 2)

#             # draw the center (x, y)-coordinates of the AprilTag
#             (cX, cY) = (int(r.center[0]), int(r.center[1]))
#             cv2.circle(screenFrame, (cX, cY), 5, (0, 0, 255), -1)

#             # draw the tag family on the image
#             tagFamily = r.tag_family.decode("utf-8")
#             cv2.putText(screenFrame, tagFamily, (ptA[0], ptA[1] - 15),
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
#             print("[INFO] tag family: {}".format(tagFamily))
        
        #Add text to frame to update displayed parameters
        eyeFrame = cv2.rectangle(eyeFrame,(0,0),(2000,60),(0,0,0),-5) #black background for white text
        eyeFrame = cv2.putText(eyeFrame, f'Min Area: {params.minArea}', (10, 15), font2, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Max Area: {params.maxArea}', (10, 55), font2, 0.5, (255,255,255), 1)

        eyeFrame = cv2.putText(eyeFrame, f'Min Circ: {params.minCircularity}', (200, 15), font2, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Max Circ: {params.maxCircularity}', (200, 55), font2, 0.5, (255,255,255), 1)

        eyeFrame = cv2.putText(eyeFrame, f'Color: {colorParam}', (400, 15), font2, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Convexity: {convexityParam}', (600, 15), font2, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Inertia: {inertiaParam}', (800, 15), font2, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Threshold: {inertiaParam}', (1000, 15), font2, 0.5, (255,255,255), 1)

        #Update Parameters
        params.minArea = 1000
        params.maxArea = 5000
        params.minCircularity = 0.5 #circularityParam/100
        params.maxCircularity = 1 #circularityParam/100
        params.minConvexity = 0
        params.maxConvexity = 1
        params.minInertiaRatio = 0 #inertiaParam/100 
        params.maxInertiaRatio = 1 #inertiaParam/100
        params.minThreshold = 0
        params.maxThreshold = 255

        

        # Create a detector with the parameters

        ver = (cv2.__version__).split('.')
        if int(ver[0]) < 3 :
            detector = cv2.SimpleBlobDetector(params)
        else : 
            detector = cv2.SimpleBlobDetector_create(params)

        keypoints = detector.detect(eyeFrame)

        points = cv2.KeyPoint_convert(keypoints)
        pointsTuple = tuple(map(tuple,points))

        #print(keypointsfirst)
        #print(keypoints[1])
        
        #check if 
        cursorSwitchOn = cv2.getTrackbarPos(cursorSwitch, image)
        
        if iter:
            root = tk.Tk()
            width = root.winfo_screenwidth()
            height = root.winfo_screenheight() #getting screen size

            (x, y) = (width//2, height//2) # ball position (at center of screen)


            pg.init()
            window = pg.display.set_mode((width, height))
            pg.display.set_caption("Calibration Window")
            print('Created window')

            ix, iy = int(x), int(y)
            circle_center = (ix,iy)
            screen_center = (width/2,height/2) 

            font = pg.font.Font('freesansbold.ttf', 18)
            text = font.render('Move your cursor to the center circle and press the letter c when done', True, pg.Color("black"))
            textRect = text.get_rect()
            # set the center of the rectangular object.
            textRect.center = (x, y+200)
            white = (255, 255, 255)


            print('Checking for key')
            found = 1
            x0 = 0
            y0 = 0

            perm_offset = 0
            print(pointsTuple)

            while True:
                window.fill(white)
                pg.draw.circle(window, pg.Color("red"), (ix, iy), 10)
                window.blit(text, textRect)
                
                _, eyeFrame = eyeCap.read()
                keypoints = detector.detect(eyeFrame)

                points = cv2.KeyPoint_convert(keypoints)
                pointsTuple = tuple(map(tuple,points))
                
                if pointsTuple:
                    xcoord = int(pointsTuple[0][0])
                    ycoord = int(pointsTuple[0][1])

                    pyautogui.moveTo(xcoord, ycoord)

                    for event in pg.event.get():
                        if pg.key.get_pressed()[pg.K_c]:
                            x0 = int(pointsTuple[0][0])
                            y0 = int(pointsTuple[0][1])
                            x_cursor, y_cursor = pg.mouse.get_pos()
                            perm_offset = (x_cursor - ix, y_cursor - iy)
                            found = 0   
                            break
                        pg.display.update()
                    if found == 0:
                            break

            print('Found offset')

            print('Call calibrate')
            if pointsTuple:
                root = tk.Tk()
                width = root.winfo_screenwidth()
                height = root.winfo_screenheight() #getting screen size

                pg.init()
                window = pg.display.set_mode((width, height))
                pg.display.set_caption("Calibration Window")


                circle_center = (width-100, height-100)
                screen_center = (width/2,height/2) 


                text = font.render('Move your cursor to the circle and press the letter w when done', True, pg.Color("black"))
                textRect = text.get_rect()
                # set the center of the rectangular object.
                textRect.center = (width/2, height/2+200)
                white = (255, 255, 255)

                found = 1
                eye_at_corner_x = 0
                eye_at_corner_y = 0
                while True:
                    _, eyeFrame = eyeCap.read()
                    keypoints = detector.detect(eyeFrame)

                    points = cv2.KeyPoint_convert(keypoints)
                    pointsTuple = tuple(map(tuple,points))
                    
                    window.fill(white)
                    pg.draw.circle(window, pg.Color("red"), (width-100,height-100), 10)
                    window.blit(text, textRect)
                    
                    if pointsTuple:
                        pyautogui.moveTo(pointsTuple[0][0] - perm_offset[0], pointsTuple[0][1] - perm_offset[1])

                    for event in pg.event.get():
                        if pg.key.get_pressed()[pg.K_w]:
                            cursor_x, cursor_y = pg.mouse.get_pos()
                            found = 0   
                            break
                        pg.display.update()
                    if found == 0:
                            break

                print(screen_center, circle_center, cursor_x, cursor_y)

                theta = angle_offset(screen_center, circle_center, (x0, y0), screen_dist)
                scale_factor = scaling(screen_center, circle_center, (cursor_x, cursor_y))
                print('Calibrated')
        
        iter = 0
        
        if pointsTuple:
            xcoord = int(pointsTuple[0][0])
            ycoord = int(pointsTuple[0][1])
            xcoord2 = ((xcoord-x0) * scale_factor[0]) + perm_offset[0]
            ycoord2 = ((ycoord-y0) * scale_factor[1]) + perm_offset[1]
            if cursorSwitchOn:
                pyautogui.moveTo(xcoord2,ycoord2)

            eyeFrame = cv2.circle(eyeFrame,(xcoord, ycoord), 5, (0,0,255), 5)



        eye = cv2.drawKeypoints(eyeFrame, keypoints, eyeFrame, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

        cv2.imshow(image, eyeFrame)
        #cv2.imshow(slf, screenFrame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()